<a href="https://colab.research.google.com/github/SSPod29/TextMining/blob/main/Assignment1Clu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis

     |████████████████████████████████| 1.7MB 4.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 9.9MB 43.6MB/s 
     |████████████████████████████████| 15.3MB 179kB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136870 sha256=870401077c70271c08eb7553f90974f18a48d1f3e50ef72c1764a6fed945a842
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0

In [2]:
import nltk
import numpy as np
import pandas as pd

In [3]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv("drive/My Drive//albums.csv")
print(df)

                             Name                                            Details
0              The Queen Is Dead   What distinguishes it as the greatest ever mad...
1                        Revolver  Far and away the best album of rock’s Phase On...
2                      Hunky Dory  If the message of ‘Changes’ was that nothing l...
3                      Is This It  That was just the set list we had been playing...
4   The Velvet Underground & Nico  The Velvet Underground & Nico has been worn sm...
..                            ...                                                ...
95         Fear Of A Black Planet  Dave Maclean, Django Django: “I remember going...
96                     The Smiths  Often overlooked due to the pedigree of the li...
97  In The Aeroplane Over The Sea  Ecstatic suicides, carrot-flower kings and sem...
98                 The Libertines  A ramshackle mess of a record with Pete’s voca...
99               Hatful Of Hollow  That was the album when The Sm

In [6]:
df.iloc[0]["Details"]

'What distinguishes it as the greatest ever made? For one thing, timelessness. It is a state-of-the-nation address which seems impervious to the passage of years. Morrissey and Marr compliment each other perfectly. It is one of those select few albums which seem to transcend its influences, working them into something singular and new.'

In [7]:
import re
from nltk.corpus import wordnet as wn 
en_stop = nltk.corpus.stopwords.words('english')
en_stop= ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<"]                  \
         +en_stop

def preprocessing_text(text):
  def cleaning_text(text):
    pattern1 = '@|%'
    text = re.sub(pattern1, '', text)    
    pattern2 = '\[[0-9 ]*\]'
    text = re.sub(pattern2, '', text)    
    pattern3 = '\([a-z ]*\)'
    text = re.sub(pattern3, '', text)    
    pattern4 = '[0-9]'
    text = re.sub(pattern4, '', text)
    return text
  
  def tokenize_text(text):
    text = re.sub('[.,]', '', text)
    return text.split()


  def lemmatize_word(word):
    # make words lower  example: Python =>python
    word=word.lower()
    
    # lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
      return lemma
  
  
  def remove_stopwords(word, stopwordset):
    if word in stopwordset:
      return None
    else:
      return word


  text = cleaning_text(text)
  tokens = tokenize_text(text)
  tokens = [lemmatize_word(word) for word in tokens]
  tokens = [remove_stopwords(word, en_stop) for word in tokens]
  tokens = [word for word in tokens if word is not None]
  return tokens
  
docs = df["Details"].values
pp_docs = [preprocessing_text(text) for text in docs]

In [8]:
def tfidf_vectorizer(docs):
  def tf(word2id, doc):
    term_counts = np.zeros(len(word2id))
    for term in word2id.keys():
      term_counts[word2id[term]] = doc.count(term)
    tf_values = list(map(lambda x: x/sum(term_counts), term_counts))
    return tf_values
  
  def idf(word2id, docs):
    idf = np.zeros(len(word2id))
    for term in word2id.keys():
      idf[word2id[term]] = np.log(len(docs) / sum([bool(term in doc) for doc in docs]))
    return idf
  
  word2id = {}
  for doc in docs:
    for w in doc:
      if w not in word2id:
        word2id[w] = len(word2id)
  
  return [[_tf*_idf for _tf, _idf in zip(tf(word2id, doc), idf(word2id, docs))] for doc in docs], word2id

In [9]:
tfidf_vector, word2id = tfidf_vectorizer(pp_docs)
print(tfidf_vector)
print(word2id.items())

[[0.1705618587402997, 0.11921762314326668, 0.11095304716866632, 0.1705618587402997, 0.1312560623653241, 0.14488974094178317, 0.1705618587402997, 0.1705618587402997, 0.1705618587402997, 0.22190609433733263, 0.1705618587402997, 0.1705618587402997, 0.10420039691703838, 0.14488974094178317, 0.1705618587402997, 0.1705618587402997, 0.1705618587402997, 0.1705618587402997, 0.02718404352148891, 0.14488974094178317, 0.11095304716866632, 0.14488974094178317, 0.10420039691703838, 0.1705618587402997, 0.07281899468047529, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [10]:
def cosine_similarity(list_a, list_b):
  inner_prod = np.array(list_a).dot(np.array(list_b))
  norm_a = np.linalg.norm(list_a)
  norm_b = np.linalg.norm(list_b)
  try:
      return inner_prod / (norm_a*norm_b)
  except ZeroDivisionError:
      return 1.0

In [11]:
def calc_cosine(vector, vector_list):
  result = {}
  for i, x in enumerate(vector_list):
    result[i] = cosine_similarity(vector, vector_list[i])
    
  return result

print("tfidf")
res = calc_cosine(tfidf_vector[0],tfidf_vector)
res

tfidf


{0: 1.0,
 1: 0.035069855419119327,
 2: 0.0010725059660411699,
 3: 0.014225006809698567,
 4: 0.030549940348384482,
 5: 0.013530578012901001,
 6: 0.06269833108188937,
 7: 0.0009069845294813098,
 8: 0.0,
 9: 0.001266640206500192,
 10: 0.0012768591487539445,
 11: 0.007844971146263178,
 12: 0.0,
 13: 0.0014417204109129068,
 14: 0.018711943039867274,
 15: 0.020220128808422957,
 16: 0.0,
 17: 0.0,
 18: 0.022917991213684444,
 19: 0.0,
 20: 0.0023821106659926983,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0014262151333865694,
 25: 0.030271262091054567,
 26: 0.0,
 27: 0.0,
 28: 0.009695736225848565,
 29: 0.009127358691770249,
 30: 0.017553722438189662,
 31: 0.0,
 32: 0.00933595955672236,
 33: 0.0012254370285797094,
 34: 0.0012120290588978327,
 35: 0.0,
 36: 0.0011754389072633224,
 37: 0.0012197320894948426,
 38: 0.09065194773304798,
 39: 0.035858864636391576,
 40: 0.0010431485812627758,
 41: 0.0,
 42: 0.0,
 43: 0.0027857449468415307,
 44: 0.04660114194511231,
 45: 0.016467174156790745,
 46: 0.0,
 47: 

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import collections

In [13]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
pre_docs = pp_docs
pre_docs = [" ".join(doc) for doc in pre_docs]
print(pre_docs[0])

vectorizer = TfidfVectorizer(max_features=200, token_pattern=u'(?u)\\b\\w+\\b' )

distinguish greatest ever made? one thing timelessness state-of-the-nation address seem impervious passage years morrissey marr compliment perfectly one select album seem transcend influence working something singular new


In [15]:
tf_idf = vectorizer.fit_transform(pre_docs)

In [37]:
num_clusters = 6
km = KMeans(n_clusters=num_clusters, random_state = 0)

In [38]:
clusters = km.fit_predict(tf_idf)

In [39]:
for doc, cls in zip(pre_docs, clusters):
    print(cls,doc)

0 distinguish greatest ever made? one thing timelessness state-of-the-nation address seem impervious passage years morrissey marr compliment perfectly one select album seem transcend influence working something singular new
0 far away best album rock’s phase one virtually every one minutes bring fresh revelation feature stylistic cultural innovation first ten minutes band achieve lifespan slacker pop psychedelia new wave dance: ‘revolver’ fountainhead we’re still drinking deep
3 message ‘changes’ wa nothing last forever it’s ironic song ha go become endure hit ‘hunky dory’ time-tested album forget glitter spider weird eyes wa bowie’s incredible song-writing gift ‘hunky dory’ convince us wa beam star
1 wa set list playing wa underneath finger feeling wa one extreme excitement felt like would succeed point would able make another record felt like really cool band playing really cool song like awesome…
0 velvet underground nico ha wear smooth years nowadays baptismal font hipness it’s one

In [43]:
num_clusters = 3
km = KMeans(n_clusters=num_clusters, random_state = 0)

In [44]:
clusters = km.fit_predict(tf_idf)

In [49]:
for doc, cls in zip(pre_docs, clusters):
    print(cls,doc)

2 distinguish greatest ever made? one thing timelessness state-of-the-nation address seem impervious passage years morrissey marr compliment perfectly one select album seem transcend influence working something singular new
2 far away best album rock’s phase one virtually every one minutes bring fresh revelation feature stylistic cultural innovation first ten minutes band achieve lifespan slacker pop psychedelia new wave dance: ‘revolver’ fountainhead we’re still drinking deep
2 message ‘changes’ wa nothing last forever it’s ironic song ha go become endure hit ‘hunky dory’ time-tested album forget glitter spider weird eyes wa bowie’s incredible song-writing gift ‘hunky dory’ convince us wa beam star
2 wa set list playing wa underneath finger feeling wa one extreme excitement felt like would succeed point would able make another record felt like really cool band playing really cool song like awesome…
0 velvet underground nico ha wear smooth years nowadays baptismal font hipness it’s one

In [22]:
import gensim
from gensim import corpora

In [23]:
dictionary = corpora.Dictionary(pp_docs)

corpus_ = [dictionary.doc2bow(doc) for doc in pp_docs]

In [24]:
print(dictionary.token2id)

{'address': 0, 'album': 1, 'compliment': 2, 'distinguish': 3, 'ever': 4, 'greatest': 5, 'impervious': 6, 'influence': 7, 'made?': 8, 'marr': 9, 'morrissey': 10, 'new': 11, 'one': 12, 'passage': 13, 'perfectly': 14, 'seem': 15, 'select': 16, 'singular': 17, 'something': 18, 'state-of-the-nation': 19, 'thing': 20, 'timelessness': 21, 'transcend': 22, 'working': 23, 'years': 24, 'achieve': 25, 'away': 26, 'band': 27, 'best': 28, 'bring': 29, 'cultural': 30, 'dance:': 31, 'deep': 32, 'drinking': 33, 'every': 34, 'far': 35, 'feature': 36, 'first': 37, 'fountainhead': 38, 'fresh': 39, 'innovation': 40, 'lifespan': 41, 'minutes': 42, 'phase': 43, 'pop': 44, 'psychedelia': 45, 'revelation': 46, 'rock’s': 47, 'slacker': 48, 'still': 49, 'stylistic': 50, 'ten': 51, 'virtually': 52, 'wave': 53, 'we’re': 54, '‘revolver’': 55, 'beam': 56, 'become': 57, 'bowie’s': 58, 'convince': 59, 'dory’': 60, 'endure': 61, 'eyes': 62, 'forever': 63, 'forget': 64, 'gift': 65, 'glitter': 66, 'go': 67, 'ha': 68, 'h

In [25]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=20,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

In [26]:
topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.007*"radiohead’s" + 0.006*"radiohead" + 0.006*"many" + 0.006*"computer’" + 0.005*"dylan’s" + 0.005*"‘ok" + 0.005*"time" + 0.005*"wa" + 0.005*"would" + 0.005*"record" + 0.003*"increase" + 0.003*"introduce" + 0.003*"song" + 0.003*"come" + 0.003*"crystallize"')
(1, '0.014*"album" + 0.013*"it’s" + 0.006*"–" + 0.005*"song" + 0.005*"debut" + 0.005*"make" + 0.005*"ever" + 0.005*"uk" + 0.005*"go" + 0.004*"back" + 0.004*"every" + 0.004*"status" + 0.004*"greatest" + 0.004*"shoes" + 0.004*"dylan’s"')
(2, '0.008*"wa" + 0.007*"would" + 0.006*"enough" + 0.006*"record" + 0.006*"instrument" + 0.006*"go" + 0.005*"it’s" + 0.005*"band" + 0.005*"sort" + 0.005*"music" + 0.005*"beatles’" + 0.005*"‘the" + 0.005*"sound" + 0.005*"like" + 0.004*"make"')
(3, '0.009*"wa" + 0.007*"album" + 0.007*"rock" + 0.007*"pop" + 0.007*"new" + 0.006*"record" + 0.005*"like" + 0.005*"one" + 0.005*"–" + 0.004*"influence" + 0.004*"song" + 0.004*"it’s" + 0.004*"track" + 0.004*"back" + 0.004*"use"')
(4, '0.010*"album" + 0.01

In [27]:
for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(16, 0.93447995)]
document ID 1:[(4, 0.9499984)]
document ID 2:[(12, 0.9486464)]
document ID 3:[(9, 0.94571084)]
document ID 4:[(9, 0.94411534)]
document ID 5:[(16, 0.95127976)]
document ID 6:[(12, 0.49433914), (15, 0.4475921)]
document ID 7:[(8, 0.96199834)]
document ID 8:[(2, 0.9472196)]
document ID 9:[(9, 0.9321412)]


In [34]:
print(ldamodel.get_document_topics(corpus_[n]))

[(9, 0.93214136)]
